In [2]:
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_curve, roc_auc_score, auc
from xgboost.sklearn import XGBClassifier
from functools import partial
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

DATA_PATH = '../data/'

/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
df = pd.read_csv(DATA_PATH + 'light_prepared_data.csv')
df.head(10)

,target,vacancy_1,vacancy_2,vacancy_3,vacancy_4,vacancy_5,vacancy_6,vacancy_7,vacancy_8,vacancy_9,...,cv_303,cv_304,cv_305,cv_306,cv_307,cv_308,cv_309,cv_310,cv_311,cv_312
0,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.01692,0.025250,-0.008520,0.03650,-0.010956,0.03540,0.019550,0.007103,0.02689,-0.03876
1,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.04938,-0.010440,-0.018020,0.04710,-0.045930,0.00759,0.013050,0.010750,0.02934,-0.01372
2,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.01761,-0.098500,-0.032840,0.03350,-0.033200,0.03310,-0.006780,0.051000,0.05340,-0.04110
3,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.01761,-0.098500,-0.032840,0.03350,-0.033200,0.03310,-0.006780,0.051000,0.05340,-0.04110
4,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.03790,-0.015850,-0.001335,0.03073,0.002558,0.03500,0.021400,0.032040,0.02356,-0.02785
5,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.05750,0.014595,-0.049870,0.05573,0.019960,0.01102,0.029950,-0.023220,0.05704,-0.06415
6,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.03412,-0.012344,-0.010000,0.02478,-0.027390,0.01962,-0.002752,-0.004390,0.03040,-0.04514
7,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.00794,0.022840,-0.005898,0.04590,-0.021710,0.04556,0.018650,0.020540,0.01800,-0.06730
8,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.02246,-0.023200,-0.006380,0.07680,-0.030490,0.04500,0.036740,0.041780,0.02174,-0.03500
9,0.0,0.07,0.003983,0.02249,-0.0743,0.00631,0.01775,-0.02373,-0.06018,0.0415,...,0.03732,-0.013960,-0.014610,0.02638,-0.018920,0.05264,0.007202,0.027040,0.02286,-0.04240


In [5]:
from sklearn.model_selection import train_test_split

X, y = df.drop(columns=['target']), df['target']
X_fulltrain, X_test, y_fulltrain, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_fulltrain, y_fulltrain, test_size=0.2, random_state=42, stratify=y_fulltrain)

X_train.head(10)

,vacancy_1,vacancy_2,vacancy_3,vacancy_4,vacancy_5,vacancy_6,vacancy_7,vacancy_8,vacancy_9,vacancy_10,...,cv_303,cv_304,cv_305,cv_306,cv_307,cv_308,cv_309,cv_310,cv_311,cv_312
33415,0.06250,0.08240,0.037000,-0.10040,-0.009476,0.015990,-0.04420,-0.02539,-0.019490,0.02250,...,0.02246,-0.023200,-0.006380,0.07680,-0.03049,0.04500,0.036740,0.041780,0.021740,-0.035000
51516,0.08936,0.08470,0.021840,-0.08923,0.014070,0.004345,-0.03280,-0.01189,-0.012540,0.04996,...,0.01312,-0.000911,-0.017230,0.07556,-0.00489,0.02467,0.012540,0.008446,0.041870,-0.034900
9774,0.01804,0.05457,0.038760,-0.07590,-0.005672,0.024030,-0.02345,-0.06168,-0.003038,0.02925,...,0.04828,-0.065800,-0.027500,0.03650,-0.06396,0.07630,0.005905,0.024750,0.040650,-0.007996
23688,0.04364,0.02667,0.004640,-0.07367,-0.039030,0.069640,-0.07600,-0.11597,0.009575,-0.01753,...,0.04590,-0.010190,-0.027400,0.07740,-0.00573,0.03041,-0.001741,0.020430,0.047030,-0.025340
21265,0.03778,0.02924,0.007484,-0.06980,-0.036200,0.055270,-0.07420,-0.10815,0.015076,-0.01324,...,0.04578,0.017580,-0.025800,0.04630,-0.04820,0.02083,0.001442,0.021330,0.050000,-0.048520
41993,0.05576,0.06570,0.034800,-0.06494,0.011810,-0.016660,-0.04105,-0.06168,0.028440,0.03912,...,0.05038,-0.016830,-0.016000,0.04324,-0.03060,0.01133,0.025040,0.006065,0.045900,-0.038820
27810,0.05630,0.03464,0.042880,-0.06976,-0.000862,0.000509,-0.05927,-0.07840,0.023800,0.01143,...,0.01036,0.018620,0.014080,0.01900,0.03244,0.03156,-0.003233,0.081800,0.006237,-0.005970
46969,0.06110,0.06930,0.030800,-0.08890,0.013490,0.024540,-0.02826,-0.02048,0.001670,0.03574,...,0.01721,-0.044430,-0.000669,0.01935,-0.03513,0.03375,0.014435,0.030440,0.039920,-0.059540
41202,0.05417,0.07750,0.030780,-0.10410,0.015820,0.007770,-0.02666,-0.02255,-0.030120,0.05112,...,0.06930,-0.026320,-0.032960,0.06793,-0.02266,0.03458,0.043400,-0.009705,0.056400,-0.032870
18459,0.09640,0.05414,0.028320,-0.08655,0.020300,0.030440,-0.04922,-0.02646,-0.012680,0.03180,...,0.02168,0.026370,-0.000685,0.05480,-0.02377,0.02441,-0.015690,0.036650,0.042000,-0.025180


In [6]:
y_train.value_counts(), y_val.value_counts(), y_test.value_counts()

(0.0    37415
 1.0     3758
 Name: target, dtype: int64,
 0.0    9354
 1.0     940
 Name: target, dtype: int64,
 0.0    11692
 1.0     1175
 Name: target, dtype: int64)

In [7]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=NPP0QK0QME8LT87B4GZ4
%env CLEARML_API_SECRET_KEY=GWiK2dXe7FnB5xaqAePS3ck5mkLa8UBLpWGJNt1vw1GHhaQKDP

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=NPP0QK0QME8LT87B4GZ4
env: CLEARML_API_SECRET_KEY=GWiK2dXe7FnB5xaqAePS3ck5mkLa8UBLpWGJNt1vw1GHhaQKDP


In [14]:
from clearml import Task

task = Task.init(project_name='Team 10 CV', task_name='XGBoost test')

model = XGBClassifier(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.1,
    # subsample=best['subsample'],
    # colsample_bytree=best['colsample_bytree'],
    eval_metric='auc',
    tree_method='hist',
    n_jobs=-1,
    use_label_encoder=False
)

model.fit(X_fulltrain, y_fulltrain, )
pred_fulltrain = model.predict(X_fulltrain)
pred_test = model.predict(X_test)

print(f'Train XGBoost accuracy: {accuracy_score(y_fulltrain, pred_fulltrain):.5f}')
print(f'Test XGBoost accuracy:  {accuracy_score(y_test, pred_test):.5f}')

print(f'Train XGBoost AUC-ROC: {roc_auc_score(y_fulltrain, pred_fulltrain):.5f}')
print(f'Test XGBoost AUC-ROC: {roc_auc_score(y_test, pred_test):.5f}')

precision_train, recall_train, _ = precision_recall_curve(y_fulltrain, pred_fulltrain)
precision, recall, _ = precision_recall_curve(y_test, pred_test)
print(f'Train XGBoost AUC-PR: {auc(recall_train, precision_train):.5f}')
print(f'Test XGBoost AUC-PR: {auc(recall, precision):.5f}')

/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Train XGBoost accuracy: 0.98304
Test XGBoost accuracy:  0.97109
Train XGBoost AUC-ROC: 0.91302
Test XGBoost AUC-ROC: 0.85357
Train XGBoost AUC-PR: 0.91372
Test XGBoost AUC-PR: 0.85023


wandb: Network error (ConnectTimeout), entering retry loop.
Retrying (Retry(total=237, connect=238, read=239, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff1d883c2b0>: Failed to establish a new connection: [Errno 65] No route to host')': /v2.23/tasks.add_or_update_artifacts
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff200f337c0>: Failed to establish a new connection: [Errno 65] No route to host')': /v2.23/tasks.get_all
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff200ae59a0>: Failed to establish a new connection: [Errno 65] No route to host')': /v2.23/tasks.get_all
Retrying (Retry(total=236, connect=237, read=239, redirect=240, status=240)) after conn

ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Retrying (Retry(total=233, connect=234, read=239, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff200ad1730>: Failed to establish a new connection: [Errno 65] No route to host')': /v2.23/tasks.add_or_update_artifacts
wandb: Network error (ConnectTimeout), entering retry loop.
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ff228b0c850>, 'Connection to api.clear.ml timed out. (connect timeout=10.0)')': /v2.23/events.add_batch
wandb: Network error (ConnectionError), entering retry loop.
Retrying (Retry(total=237, connect=238, read=239, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff1d8ce0460>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')': /v2.23/events.add_bat

In [1]:
! python3 -m wandb login XXX

wandb: Appending key for api.wandb.ai to your netrc file: /Users/dangl/.netrc


In [9]:
import wandb

search_space = {
    'n_estimators': hp.qnormal('n_estimators', 500, 200, 1),
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'subsample': hp.uniform('subsample', 0.4, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 0.5),
}

def objective(params, X_train, y_train, X_val, y_val):
    wandb.init(project='GOP HW2', entity='dangl', name='hyperopt search', group='xgboost', config=params)

    model = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        eval_metric='auc',
        tree_method='hist',
        n_jobs=-1,
        use_label_encoder=False
    )

    model.fit(X_train, y_train, verbose=False, early_stopping_rounds=100, eval_set=[(X_val, y_val)])

    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)

    precision_train, recall_train, _ = precision_recall_curve(y_train, pred_train)
    precision, recall, _ = precision_recall_curve(y_val, pred_val)

    log = {
        'Train accuracy': accuracy_score(y_train, pred_train),
        'Val accuracy': accuracy_score(y_val, pred_val),
        'Train AUC-ROC': roc_auc_score(y_train, pred_train),
        'Val AUC-ROC': roc_auc_score(y_val, pred_val),
        'Train AUC-PR': auc(recall_train, precision_train),
        'Val AUC-PR': auc(recall, precision),
    }

    wandb.log(log)

    return {'loss': -roc_auc_score(y_val, pred_val), 'params': params, 'status': STATUS_OK}

trials = Trials()
best = fmin(  
    fn=partial(objective, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val),
    space=search_space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials,
    rstate=np.random.default_rng(42),
    show_progressbar=True
)

print(best)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

Train AUC-PR,0.99359
Train AUC-ROC,0.99295
Train accuracy,0.99871
Val AUC-PR,0.88578
Val AUC-ROC,0.88531
Val accuracy,0.97766


/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):



  3%|▎         | 1/30 [00:54<26:22, 54.56s/trial, best loss: -0.8853149636746596]

Train AUC-PR,0.99359
Train AUC-ROC,0.99295
Train accuracy,0.99871
Val AUC-PR,0.88578
Val AUC-ROC,0.88531
Val accuracy,0.97766


...

 97%|█████████▋| 29/30 [18:07<00:51, 51.46s/trial, best loss: -0.8991955881884642]

Train AUC-PR,0.95206
Train AUC-ROC,0.95036
Train accuracy,0.99048
Val AUC-PR,0.85544
Val AUC-ROC,0.85781
Val accuracy,0.97202


/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):



100%|██████████| 30/30 [18:46<00:00, 37.56s/trial, best loss: -0.8991955881884642]
{'colsample_bytree': 0.27811458697033753, 'learning_rate': 0.09264749650673572, 'max_depth': 5.0, 'n_estimators': 662.0, 'subsample': 0.7384906910206007}


wandb: Network error (ConnectionError), entering retry loop.


In [1]:
## found
best = {'colsample_bytree': 0.27811458697033753, 'learning_rate': 0.09264749650673572, 'max_depth': 5.0, 'n_estimators': 662.0, 'subsample': 0.7384906910206007}

In [7]:
model = XGBClassifier(
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    learning_rate=best['learning_rate'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    eval_metric='auc',
    tree_method='hist',
    n_jobs=-1,
    use_label_encoder=False
)

model.fit(X_fulltrain, y_fulltrain, )
pred_fulltrain = model.predict(X_fulltrain)
pred_test = model.predict(X_test)

print(f'Train XGBoost accuracy: {accuracy_score(y_fulltrain, pred_fulltrain):.5f}')
print(f'Test XGBoost accuracy:  {accuracy_score(y_test, pred_test):.5f}')

print(f'Train XGBoost AUC-ROC: {roc_auc_score(y_fulltrain, pred_fulltrain):.5f}')
print(f'Test XGBoost AUC-ROC: {roc_auc_score(y_test, pred_test):.5f}')

precision_train, recall_train, _ = precision_recall_curve(y_fulltrain, pred_fulltrain)
precision, recall, _ = precision_recall_curve(y_test, pred_test)
print(f'Train XGBoost AUC-PR: {auc(recall_train, precision_train):.5f}')
print(f'Test XGBoost AUC-PR: {auc(recall, precision):.5f}')

/Users/dangl/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Train XGBoost accuracy: 0.99883
Test XGBoost accuracy:  0.98073
Train XGBoost AUC-ROC: 0.99390
Test XGBoost AUC-ROC: 0.90442
Train XGBoost AUC-PR: 0.99416
Test XGBoost AUC-PR: 0.90088


In [8]:
model.save_model('../trained_models/xgboost_best.json')